***Import Necessary Libraries***

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import sidetable
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import seaborn as seabornInstance

***Load the dataset***

In [3]:
df=pd.read_csv('/content/country_vaccinations.csv')

In [4]:
df.head()


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


***Data Preprocessing***

In [5]:
indices=df.loc[df['total_vaccinations']< (df['people_vaccinated'] + df['people_fully_vaccinated'] )].index
indices

Int64Index([  265,   308,   321,   323,   325,   329,   331,   336,   338,
              339,
            ...
            25267, 25270, 25272, 25275, 25341, 25674, 25677, 25681, 25686,
            25695],
           dtype='int64', length=2096)

In [6]:
df.loc[df.index.isin(indices),'total_vaccinations'] = df.loc[df.index,'people_vaccinated']+df.loc[df.index,'people_fully_vaccinated']

In [7]:
indices=df.loc[ (df['total_vaccinations']< df['people_fully_vaccinated']) & df['people_vaccinated'].isnull()].index
indices

Int64Index([], dtype='int64')

In [8]:
df.loc[df.index.isin(indices),'total_vaccinations'] = df.loc[df.index,'people_fully_vaccinated']

In [9]:
indices=df.loc[df['total_vaccinations']> (df['people_vaccinated'] + df['people_fully_vaccinated'] )].index
indices

Int64Index([  691,   694,   695,   697,   698,   699,   702,   704,   705,
              706,
            ...
            26785, 26792, 26799, 26806, 26813, 26820, 26827, 26834, 26841,
            26848],
           dtype='int64', length=4157)

In [11]:
df.loc[df.index.isin(indices),'total_vaccinations'] = df.loc[df.index,'people_vaccinated']+df.loc[df.index,'people_fully_vaccinated']
df.isnull().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                     13859
people_vaccinated                      14665
people_fully_vaccinated                15532
daily_vaccinations_raw                 16487
daily_vaccinations                        89
total_vaccinations_per_hundred         13859
people_vaccinated_per_hundred          14665
people_fully_vaccinated_per_hundred    15532
daily_vaccinations_per_million            89
vaccines                                   0
source_name                                0
source_website                             0
dtype: int64

In [12]:
df_N=df.dropna()
df_N.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
94,Afghanistan,AFG,2021-05-27,593313.0,479574.0,113739.0,2859.0,6487.0,1.49,1.20,0.29,163.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
101,Afghanistan,AFG,2021-06-03,630305.0,481800.0,148505.0,4015.0,5285.0,1.58,1.21,0.37,133.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
339,Afghanistan,AFG,2022-01-27,8386212.0,4517380.0,3868832.0,6868.0,9802.0,12.76,11.34,9.71,246.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
433,Albania,ALB,2021-02-18,3049.0,2438.0,611.0,1348.0,254.0,0.11,0.08,0.02,88.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",Ministry of Health,https://shendetesia.gov.al/vaksinimi-anticovid...
515,Albania,ALB,2021-05-11,622507.0,440921.0,181586.0,9548.0,12160.0,21.67,15.35,6.32,4233.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",Ministry of Health,https://shendetesia.gov.al/vaksinimi-anticovid...


In [13]:
indices= df.loc[(df['daily_vaccinations_raw'].isnull()) & (df['daily_vaccinations'].notnull())].index
indices

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            26871, 26872, 26873, 26874, 26875, 26876, 26877, 26878, 26879,
            26880],
           dtype='int64', length=16398)

In [14]:
df.loc[df.index.isin(indices),'daily_vaccinations_raw'] = df.loc[df.index,'daily_vaccinations']

In [15]:
indices= df.loc[(df['total_vaccinations'].isnull()) & (df['people_vaccinated'].notnull()) & (df['people_fully_vaccinated'].notnull())].index
indices

Int64Index([10282, 10568, 10585, 12376, 13368], dtype='int64')

In [16]:
df.loc[df.index.isin(indices),'total_vaccinations'] = df.loc[df.index,'people_vaccinated']+df.loc[df.index,'people_fully_vaccinated']
indices= df.loc[(df['total_vaccinations']>0) & (df['people_vaccinated'].notnull()) & (df['people_fully_vaccinated'].isnull())].index
indices

Int64Index([    6,    22,    44,    59,   190,   220,   396,   397,   398,
              399,
            ...
            26090, 26097, 26129, 26176, 26178, 26179, 26183, 26204, 26225,
            26526],
           dtype='int64', length=1073)

In [17]:
df.loc[df.index.isin(indices),'people_vaccinated'] = df.loc[df.index,'total_vaccinations']-df.loc[df.index,'people_fully_vaccinated']

In [18]:
df['Prev_vacc'] = df['total_vaccinations'].shift()
df['Prev_code'] = df['iso_code'].shift()
indices=df.loc[(df['total_vaccinations'].isnull()) & (df['Prev_vacc'].notnull()) & (df['daily_vaccinations_raw'].notnull()) & (df['Prev_code'] == df['iso_code'])].index
while (len(indices)>0):
   df.loc[df.index.isin(indices),'total_vaccinations'] = df.loc[df.index,'Prev_vacc']+df.loc[df.index,'daily_vaccinations_raw']
   df['Prev_vacc'] = df['total_vaccinations'].shift()
   indices=df.loc[(df['total_vaccinations'].isnull()) & (df['Prev_vacc'].notnull()) & (df['daily_vaccinations_raw'].notnull())  & (df['Prev_code'] == df['iso_code'])].index

In [19]:


df['Prev_vacc_full'] = df['people_fully_vaccinated'].shift()
indices=df.loc[(df['people_fully_vaccinated'].isnull()) & (df['Prev_vacc_full'].notnull())  & (df['Prev_code'] == df['iso_code'])].index
while (len(indices)>0):
   df.loc[df.index.isin(indices),'people_fully_vaccinated'] = df.loc[df.index,'Prev_vacc_full']
   df['Prev_vacc_full'] = df['people_fully_vaccinated'].shift()
   indices=df.loc[(df['people_fully_vaccinated'].isnull()) & (df['Prev_vacc_full'].notnull())  & (df['Prev_code'] == df['iso_code'])].index
indices= df.loc[(df['total_vaccinations']>0) & (df['people_vaccinated'].isnull()) & (df['people_fully_vaccinated'].notnull())].index
indices
df.loc[df.index.isin(indices),'people_vaccinated'] = df.loc[df.index,'total_vaccinations']-df.loc[df.index,'people_fully_vaccinated']
df.isnull().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                        19
people_vaccinated                       3529
people_fully_vaccinated                 3617
daily_vaccinations_raw                    89
daily_vaccinations                        89
total_vaccinations_per_hundred         13859
people_vaccinated_per_hundred          14665
people_fully_vaccinated_per_hundred    15532
daily_vaccinations_per_million            89
vaccines                                   0
source_name                                0
source_website                             0
Prev_vacc                                 20
Prev_code                                  1
Prev_vacc_full                          3618
dtype: int64

In [20]:
df['day'] =pd.DatetimeIndex(df['date']).day
df['month'] = pd.DatetimeIndex(df['date']).month
#df['year'] = pd.DatetimeIndex(df['date']).year
df['week_of_year'] =pd.DatetimeIndex(df['date']).weekofyear
df['day_of_year'] =pd.DatetimeIndex(df['date']).dayofyear
df['day_of_week'] =pd.DatetimeIndex(df['date']).dayofweek

<ipython-input-20-ea7fd89666bd>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['week_of_year'] =pd.DatetimeIndex(df['date']).weekofyear


In [21]:
df.loc[df['iso_code']=='TUN']

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,...,source_name,source_website,Prev_vacc,Prev_code,Prev_vacc_full,day,month,week_of_year,day_of_year,day_of_week


In [22]:
df['Prev_total_vaccinations_per_hundred'] = df['total_vaccinations_per_hundred'].shift()

In [23]:
indices=df.loc[(df['total_vaccinations_per_hundred'].isnull()) & (df['Prev_total_vaccinations_per_hundred'].notnull())  & (df['Prev_code'] == df['iso_code'])].index
while (len(indices)>0):
   df.loc[df.index.isin(indices),'total_vaccinations_per_hundred'] = df.loc[df.index,'Prev_total_vaccinations_per_hundred']
   df['Prev_total_vaccinations_per_hundred'] = df['total_vaccinations_per_hundred'].shift()
   indices=df.loc[(df['total_vaccinations_per_hundred'].isnull()) & (df['Prev_total_vaccinations_per_hundred'].notnull())  & (df['Prev_code'] == df['iso_code'])].index

In [24]:
df['Prev_people_vaccinated_per_hundred'] = df['people_vaccinated_per_hundred'].shift()
indices=df.loc[(df['people_vaccinated_per_hundred'].isnull()) & (df['Prev_people_vaccinated_per_hundred'].notnull())  & (df['Prev_code'] == df['iso_code'])].index
while (len(indices)>0):
   df.loc[df.index.isin(indices),'people_vaccinated_per_hundred'] = df.loc[df.index,'Prev_people_vaccinated_per_hundred']
   df['Prev_people_vaccinated_per_hundred'] = df['people_vaccinated_per_hundred'].shift()
   indices=df.loc[(df['people_vaccinated_per_hundred'].isnull()) & (df['Prev_people_vaccinated_per_hundred'].notnull())  & (df['Prev_code'] == df['iso_code'])].index

In [25]:
df.isnull().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                        19
people_vaccinated                       3529
people_fully_vaccinated                 3617
daily_vaccinations_raw                    89
daily_vaccinations                        89
total_vaccinations_per_hundred            19
people_vaccinated_per_hundred            201
people_fully_vaccinated_per_hundred    15532
daily_vaccinations_per_million            89
vaccines                                   0
source_name                                0
source_website                             0
Prev_vacc                                 20
Prev_code                                  1
Prev_vacc_full                          3618
day                                        0
month                                      0
week_of_year                               0
day_of_year                                0
day_of_wee

In [26]:
df['Prev_people_fully_vaccinated_per_hundred'] = df['people_fully_vaccinated_per_hundred'].shift()
indices=df.loc[(df['people_fully_vaccinated_per_hundred'].isnull()) & (df['Prev_people_fully_vaccinated_per_hundred'].notnull())  & (df['Prev_code'] == df['iso_code'])].index
while (len(indices)>0):
   df.loc[df.index.isin(indices),'people_fully_vaccinated_per_hundred'] = df.loc[df.index,'Prev_people_fully_vaccinated_per_hundred']
   df['Prev_people_fully_vaccinated_per_hundred'] = df['people_fully_vaccinated_per_hundred'].shift()
   indices=df.loc[(df['people_fully_vaccinated_per_hundred'].isnull()) & (df['Prev_people_fully_vaccinated_per_hundred'].notnull())  & (df['Prev_code'] == df['iso_code'])].index
df.isnull().sum()

country                                        0
iso_code                                       0
date                                           0
total_vaccinations                            19
people_vaccinated                           3529
people_fully_vaccinated                     3617
daily_vaccinations_raw                        89
daily_vaccinations                            89
total_vaccinations_per_hundred                19
people_vaccinated_per_hundred                201
people_fully_vaccinated_per_hundred         3617
daily_vaccinations_per_million                89
vaccines                                       0
source_name                                    0
source_website                                 0
Prev_vacc                                     20
Prev_code                                      1
Prev_vacc_full                              3618
day                                            0
month                                          0
week_of_year        

In [27]:
df.loc[(df['daily_vaccinations_raw'].isnull()) & (df['daily_vaccinations'].isnull())]

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,...,Prev_code,Prev_vacc_full,day,month,week_of_year,day_of_year,day_of_week,Prev_total_vaccinations_per_hundred,Prev_people_vaccinated_per_hundred,Prev_people_fully_vaccinated_per_hundred
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,...,None,NaN,22,2,8,53,0,NaN,NaN,NaN
394,Albania,ALB,2021-01-10,0.0,0.0,NaN,NaN,NaN,0.00,0.00,...,AFG,4420127.0,10,1,1,10,6,14.44,12.76,11.10
833,Algeria,DZA,2021-01-29,0.0,0.0,NaN,NaN,NaN,0.00,0.00,...,ALB,1215199.0,29,1,4,29,4,95.87,44.52,42.30
1238,Andorra,AND,2021-01-25,576.0,NaN,NaN,NaN,NaN,0.74,0.74,...,DZA,6110712.0,25,1,4,25,0,30.72,16.72,13.70
1651,Angola,AGO,2021-03-01,0.0,0.0,NaN,NaN,NaN,0.00,0.00,...,AND,53367.0,1,3,9,60,0,196.50,74.74,68.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25342,Ethiopia,ETH,2021-04-08,430000.0,NaN,NaN,NaN,NaN,0.36,0.36,...,SWZ,318360.0,8,4,14,98,3,43.45,33.05,27.16
25696,Faeroe Islands,FRO,2021-01-29,3994.0,3993.0,1.0,NaN,NaN,8.14,8.14,...,ETH,20958931.0,29,1,4,29,4,24.92,20.77,17.78
26082,Falkland Islands,FLK,2021-02-07,0.0,0.0,NaN,NaN,NaN,0.00,0.00,...,FRO,40895.0,7,2,5,38,6,211.80,85.04,83.37
26149,Fiji,FJI,2021-03-17,0.0,0.0,NaN,NaN,NaN,0.00,0.00,...,FLK,1775.0,17,3,11,76,2,124.91,74.60,50.31


In [28]:
indices=df.loc[(df['total_vaccinations']>0) & (df['Prev_vacc'].notnull()) & (df['daily_vaccinations_raw'].isnull()) &  (df['Prev_code'] == df['iso_code']) ].index
indices

Int64Index([], dtype='int64')

In [29]:
df.loc[df.index.isin(indices),'daily_vaccinations_raw'] = df.loc[df.index,'total_vaccinations']-df.loc[df.index,'Prev_vacc']
indices=df.loc[(df['total_vaccinations'].isnull()) & (df['people_vaccinated'].notnull()) & (df['people_fully_vaccinated'].notnull())].index
indices

Int64Index([], dtype='int64')

In [30]:
df.loc[df.index.isin(indices),'total_vaccinations'] = df.loc[df.index,'people_vaccinated']+df.loc[df.index,'people_fully_vaccinated']
df.isnull().sum()

country                                        0
iso_code                                       0
date                                           0
total_vaccinations                            19
people_vaccinated                           3529
people_fully_vaccinated                     3617
daily_vaccinations_raw                        89
daily_vaccinations                            89
total_vaccinations_per_hundred                19
people_vaccinated_per_hundred                201
people_fully_vaccinated_per_hundred         3617
daily_vaccinations_per_million                89
vaccines                                       0
source_name                                    0
source_website                                 0
Prev_vacc                                     20
Prev_code                                      1
Prev_vacc_full                              3618
day                                            0
month                                          0
week_of_year        

In [31]:
indices=df.loc[(df['total_vaccinations']==0) & df['daily_vaccinations_raw'].isnull() & df['daily_vaccinations'].isnull()].index
indices

Int64Index([    0,   394,   833,  1651,  2041,  2456,  4862,  5299,  6133,
             6968,  7870,  8576,  9016,  9934, 10287, 11395, 12645, 13058,
            13873, 14247, 18077, 20485, 21640, 22048, 22887, 23308, 24986,
            26082, 26149],
           dtype='int64')

In [32]:
df.loc[df.index.isin(indices),'daily_vaccinations_raw'] = df['total_vaccinations']
df.loc[df.index.isin(indices),'daily_vaccinations'] = df['total_vaccinations']
indices=df.loc[(df['total_vaccinations']==0) & df['daily_vaccinations_raw']==0 & df['daily_vaccinations'].isnull()].index
indices


Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            26871, 26872, 26873, 26874, 26875, 26876, 26877, 26878, 26879,
            26880],
           dtype='int64', length=26881)

In [33]:
df.loc[df.index.isin(indices),'daily_vaccinations'] = df['total_vaccinations']
indices=df.loc[(df['total_vaccinations']==0) & df['daily_vaccinations']==0 & df['daily_vaccinations_raw'].isnull()].index
indices

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            26871, 26872, 26873, 26874, 26875, 26876, 26877, 26878, 26879,
            26880],
           dtype='int64', length=26881)

In [34]:
df.loc[df.index.isin(indices),'daily_vaccinations_raw'] = df['total_vaccinations']
df.loc[(df['daily_vaccinations'].isnull())]

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,...,Prev_code,Prev_vacc_full,day,month,week_of_year,day_of_year,day_of_week,Prev_total_vaccinations_per_hundred,Prev_people_vaccinated_per_hundred,Prev_people_fully_vaccinated_per_hundred
18924,Cuba,CUB,2021-05-12,NaN,70000.0,NaN,NaN,NaN,NaN,0.62,...,HRV,2237864.0,12,5,19,132,2,127.99,56.61,54.83
18925,Cuba,CUB,2021-05-13,NaN,NaN,NaN,NaN,NaN,NaN,0.62,...,CUB,NaN,13,5,19,133,3,NaN,0.62,NaN
18926,Cuba,CUB,2021-05-14,NaN,NaN,NaN,NaN,NaN,NaN,0.62,...,CUB,NaN,14,5,19,134,4,NaN,0.62,NaN
18927,Cuba,CUB,2021-05-15,NaN,NaN,NaN,NaN,NaN,NaN,0.62,...,CUB,NaN,15,5,19,135,5,NaN,0.62,NaN
18928,Cuba,CUB,2021-05-16,NaN,455908.0,NaN,NaN,NaN,NaN,4.03,...,CUB,NaN,16,5,19,136,6,NaN,0.62,NaN
18929,Cuba,CUB,2021-05-17,NaN,540002.0,NaN,NaN,NaN,NaN,4.77,...,CUB,NaN,17,5,20,137,0,NaN,4.03,NaN
18930,Cuba,CUB,2021-05-18,NaN,612376.0,NaN,NaN,NaN,NaN,5.41,...,CUB,NaN,18,5,20,138,1,NaN,4.77,NaN
18931,Cuba,CUB,2021-05-19,NaN,NaN,NaN,NaN,NaN,NaN,5.41,...,CUB,NaN,19,5,20,139,2,NaN,5.41,NaN
18932,Cuba,CUB,2021-05-20,NaN,717997.0,NaN,NaN,NaN,NaN,6.34,...,CUB,NaN,20,5,20,140,3,NaN,5.41,NaN
18933,Cuba,CUB,2021-05-21,NaN,NaN,NaN,NaN,NaN,NaN,6.34,...,CUB,NaN,21,5,20,141,4,NaN,6.34,NaN


In [35]:
df.isnull().sum()

country                                        0
iso_code                                       0
date                                           0
total_vaccinations                            19
people_vaccinated                           3529
people_fully_vaccinated                     3617
daily_vaccinations_raw                        19
daily_vaccinations                            19
total_vaccinations_per_hundred                19
people_vaccinated_per_hundred                201
people_fully_vaccinated_per_hundred         3617
daily_vaccinations_per_million                89
vaccines                                       0
source_name                                    0
source_website                                 0
Prev_vacc                                     20
Prev_code                                      1
Prev_vacc_full                              3618
day                                            0
month                                          0
week_of_year        

In [36]:
df=df.drop(["country","date","Prev_vacc","Prev_code","Prev_vacc_full","Prev_total_vaccinations_per_hundred","Prev_people_vaccinated_per_hundred","Prev_people_fully_vaccinated_per_hundred"],axis=1)

In [37]:
df.loc[df['iso_code']=='TUN'][["total_vaccinations","people_vaccinated","people_fully_vaccinated","total_vaccinations_per_hundred","people_vaccinated_per_hundred","people_fully_vaccinated_per_hundred","daily_vaccinations_per_million"]]

,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million


In [38]:
indices=df.loc[(df['people_fully_vaccinated']==0) & df['people_fully_vaccinated_per_hundred'].isnull()].index
indices

Int64Index([], dtype='int64')

In [39]:
df.loc[(df['daily_vaccinations'].isnull()) | (df['people_vaccinated'].isnull()) ]

,iso_code,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,day,month,week_of_year,day_of_year,day_of_week
1,AFG,1367.0,NaN,NaN,1367.0,1367.0,0.0,0.0,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,23,2,8,54,1
2,AFG,2734.0,NaN,NaN,2734.0,2734.0,0.0,0.0,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,24,2,8,55,2
3,AFG,4101.0,NaN,NaN,4101.0,4101.0,0.0,0.0,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,25,2,8,56,3
4,AFG,5468.0,NaN,NaN,5468.0,5468.0,0.0,0.0,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,26,2,8,57,4
5,AFG,6835.0,NaN,NaN,6835.0,6835.0,0.0,0.0,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,27,2,8,58,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26535,FIN,39955.0,NaN,NaN,39955.0,39955.0,0.0,0.0,NaN,799.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",Finnish Institute for Health and Welfare,https://sampo.thl.fi/pivot/prod/en/vaccreg/cov...,12,1,2,12,1
26536,FIN,44387.0,NaN,NaN,44387.0,44387.0,0.0,0.0,NaN,799.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",Finnish Institute for Health and Welfare,https://sampo.thl.fi/pivot/prod/en/vaccreg/cov...,13,1,2,13,2
26537,FIN,48819.0,NaN,NaN,48819.0,48819.0,0.0,0.0,NaN,799.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",Finnish Institute for Health and Welfare,https://sampo.thl.fi/pivot/prod/en/vaccreg/cov...,14,1,2,14,3
26538,FIN,53251.0,NaN,NaN,53251.0,53251.0,0.0,0.0,NaN,799.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech",Finnish Institute for Health and Welfare,https://sampo.thl.fi/pivot/prod/en/vaccreg/cov...,15,1,2,15,4


In [40]:
df.isnull().sum()

iso_code                                  0
total_vaccinations                       19
people_vaccinated                      3529
people_fully_vaccinated                3617
daily_vaccinations_raw                   19
daily_vaccinations                       19
total_vaccinations_per_hundred           19
people_vaccinated_per_hundred           201
people_fully_vaccinated_per_hundred    3617
daily_vaccinations_per_million           89
vaccines                                  0
source_name                               0
source_website                            0
day                                       0
month                                     0
week_of_year                              0
day_of_year                               0
day_of_week                               0
dtype: int64

In [41]:
df['vaccines'].unique()

array(['Johnson&Johnson, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac, Sputnik V',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca', 'Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V',
       'CanSino, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'Moderna, Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac, Sputnik V',
       'Pfizer/BioNTech',
       'Johnson&Johnson, Moderna, Novavax, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Johnson&Johnson, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik Light, Sputnik V',
       'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinovac',
       'Oxford/AstraZeneca, Pfizer/BioNTech, S